# Data Cleaning Listado de Listado de Beneficiarios 2019

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. Beneficiarios2019 -> Dataset con el listado de TODOS los beneficiarios.
2. Estados_Beneficiarios 2019 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario ->
4. Dataset_Inegi ->

In [ ]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [ ]:
# Definición de funciones

def load_datasets(directory):
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    # Read each CSV file and store the DataFrame in a list
    dataframes = [pd.read_csv(file, encoding='cp1252', index_col=0, skiprows=1) for file in csv_files]

    # Concatenate all DataFrames in the list
    merged_df = pd.concat(dataframes, join='inner', ignore_index=True)

    return merged_df


def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text)
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura de los datos

### Lectura del dataset del INEGI

In [ ]:
path_dataset_inegi = '../../data/dataset_inegi.csv'
dataset_inegi = pd.read_csv(path_dataset_inegi, encoding='cp1252', dtype={'CVE_ENT': str, 'CVE_MUN': str})

### Lectura del listado de Beneficiarios 2019

In [ ]:
Beneficiarios_2019 = 

# 2. Limpieza de los datos

### 2.1 INEGI

In [ ]:
# Revisamos las columnas del dataset
dataset_inegi.columns

In [ ]:
# Revisamos las primeras filas del dataset
dataset_inegi.head()

In [ ]:
# Eliminamos las columnas que no son de interés
COLUMNS_TO_DROP = ['MAPA', 'Estatus', 'NOM_ABR', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
                   'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
                   'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS']
dataset_inegi = dataset_inegi.drop(COLUMNS_TO_DROP, axis=1)

In [ ]:
# Las claves de entidad y municipio serán tratadas numéricamente en la limpieza aunque posteriormente se les asignará el tipo de cadena de texto para tener el estándar.
dataset_inegi.dtypes

In [ ]:
# Revisamos la cantidad de filas y columnas del dataset
print("Shape of dataset_inegi: ", dataset_inegi.shape)

dataset_inegi_clean = dataset_inegi.drop_duplicates()
print("Shape of dataset_inegi_clean: ", dataset_inegi_clean.shape)

A partir de aquí seguimos trabajando con el listado de Estados y Municipios limpio de Inegi (sin repetir) "dataset_inegi_clean".

In [ ]:
# Revisamos las primeras filas del dataset con las columnas seleccionadas
dataset_inegi_clean.head()

In [ ]:
# Revisamos las últimas filas del dataset con las columnas seleccionadas
dataset_inegi_clean.tail()

In [ ]:
dataset_inegi_clean.info()

In [ ]:
print("Los valores únicos en cada columna son:\n", dataset_inegi_clean.nunique())

In [ ]:
# Creamos una columna con la clave única por municipio

dataset_inegi_clean['CVE_MUN_Unique'] = dataset_inegi_clean['CVE_ENT'].astype(str) + '-' + dataset_inegi_clean[
    'CVE_MUN'].astype(str)

dataset_inegi_clean.head()

In [ ]:
dataset_inegi_clean.tail()

#### 2.1.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los datos.

In [ ]:
# Estandarizamos la limpieza de los datos
dataset_inegi_clean['NOM_ENT_Clean'] = dataset_inegi_clean['NOM_ENT'].apply(clean_text)
dataset_inegi_clean['NOM_MUN_Clean'] = dataset_inegi_clean['NOM_MUN'].apply(clean_text)

## 2.2 Beneficiarios 2019

### Creación de Estados_Beneficiarios2019
Este dataset es una versión de Beneficiarios2019 pero más ligera y sin repeticiones.

In [ ]:
# Seleccionar solo las dos primeras columnas
Estados_Beneficiarios2019 = Beneficiarios_2019[['ESTADO', 'MUNICIPIO']]

# Obtener las filas únicas
Estados_Beneficiarios2019 = Beneficiarios_2019.drop_duplicates()

In [ ]:
# Estandarización de nombres de Estados y Municipios

In [ ]:
# Estandarizamos la limpieza de los datos
Estados_Beneficiarios2019['ESTADO_Clean'] = Estados_Beneficiarios2019['ESTADO'].apply(clean_text)
Estados_Beneficiarios2019['MUNICIPIO_Clean'] = Estados_Beneficiarios2019['MUNICIPIO'].apply(clean_text)

In [ ]:
# Valores únicos y la cantidad de cada columna
# Obtener estadísticas descriptivas para todas las variables

descriptive_stats = Beneficiarios_2019.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

# 3. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2019

El objetivo de esta sección es crear un diccionario de códigos según INEGI para los municipios Listado_beneficiarios2019. Para ello haremos un Left join entre Estados_Beneficiarios2019 y dataset_inegi_clean.

### 3.1 Creamos las columnas clave

Crearemos las columnas clave concatenando [Estado_limpio]-[Municipio_limpio] en **ambos datasets**

In [ ]:
# INEGI
dataset_inegi_clean["NOM_ENT_Clean"] = dataset_inegi_clean["NOM_ENT_Clean"].astype(str)
dataset_inegi_clean["NOM_MUN_Clean"] = dataset_inegi_clean["NOM_MUN_Clean"].astype(str)

dataset_inegi_clean["KEY_inegi"] = dataset_inegi_clean["NOM_ENT_Clean"] + "-" + dataset_inegi_clean["NOM_MUN_Clean"]

In [ ]:
# Listado_Beneficiarios2019

### 3.1 Left join

Creamos el diccionario.

In [ ]:
Estados_productores = Estados_productores.drop(['ESTADO', 'MUNICIPIO'], axis=1)
Estados_productores = Estados_productores.drop_duplicates()
Estados_productores.shape

In [ ]:

# Crear una función para encontrar la mejor coincidencia difusa con límites entre 90 y 100 de coincidencia


def fuzzy_merge_beneficiarios2019(df_inegi, df_prod, key1, key2, threshold=90, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_prod[key2].tolist()

    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_inegi[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))

    # Crear una columna con las mejores coincidencias
    df_inegi['best_match'] = [match[0] if match else None for match in matches]
    df_inegi['match_score'] = [match[1] if match else None for match in matches]

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_inegi, df_prod, left_on='best_match', right_on=key2, how='inner',
                         suffixes=('_inegi', '_prod'))
    
    return df_merged


In [ ]:
# Aplicar la función de coincidencia difusa
diccionario = fuzzy_merge(dataset_inegi_clean, Estados_productores, 'KEY_inegi', 'KEY_prod')
diccionario.drop_duplicates(subset=['KEY_inegi'], inplace=True)

# Mostrar el resultado
diccionario.columns

In [ ]:
diccionario.to_csv('../../data/merged_dataset.csv', index=False)

## 3.2 Listado beneficiarios2019

Esta sección se encarga de completar el listado original de Productores Autorizados con los nombre corregido de INEGI usando el diccionario.

In [ ]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join
listado_productores['ESTADO_Clean'] = listado_productores['ESTADO'].apply(clean_text)
listado_productores['MUNICIPIO_Clean'] = listado_productores['MUNICIPIO'].apply(clean_text)
listado_productores['Estado-mun-KEY'] = listado_productores['ESTADO_Clean'].astype(str) + '-' + listado_productores[
    'MUNICIPIO_Clean'].astype(str)

diccionario_Sin_VC = diccionario[diccionario["NOM_ENT"] != "Veracruz de Ignacio de la Llave"]

In [ ]:
listado_productores.shape

In [ ]:
diccionario_Sin_VC2 = diccionario_Sin_VC[['CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN', 'CVE_MUN_Unique',
       'KEY_inegi', 'best_match', 'match_score', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'KEY_prod']]

diccionario_Sin_VC.shape

In [ ]:
diccionario_Sin_VC.to_csv('../../data/Diccionario_autom.csv', index=False)
diccionario_Sin_VC.columns

In [ ]:
# Lectura del diccionario manipulado
diccionario_manipulado = pd.read_csv('../../data/Diccionario_manual.csv')

In [ ]:
# Hacer el join
listado_productores_complete = pd.merge(listado_productores, diccionario_manipulado, left_on="Estado-mun-KEY",
                                        right_on="KEY_prod", how='left', suffixes=('_prod', '_inegi'))

In [ ]:
listado_productores_complete[['CVE_ENT', 'CVE_MUN']] = listado_productores_complete['CVE_MUN_Unique'].str.split('-',
                                                                                                                expand=True)


In [ ]:
listado_productores_complete.columns

In [ ]:
# Seleccionamos las columnas que nos interesan
listado_productores_complete = listado_productores_complete[
    ['ESTADO', 'MUNICIPIO', 'ACUSE', 'APELLIDO PATERNO', 'APELLIDO MATERNO',
     'NOMBRE (S)', 'PAQUETE', 'KEY_inegi', 'NOM_ENT', 'NOM_MUN', 'CVE_ENT', 'CVE_MUN']]

In [ ]:
# Revisamos el dataset
print(listado_productores_complete.shape)
print(listado_productores_complete.columns)
print(listado_productores_complete.head())

In [ ]:
listado_productores_complete.to_csv('../../data/listado_productores_complete.csv', index=False)